In [ ]:
import os 
import pandas as pd 
import tensorflow as tf 
import time 

from sklearn.model_selection import train_test_split
from sklearn  import feature_selection
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline


from sklearn.ensemble import RandomForestClassifier

In [ ]:
def convert_data(a, b):
    
    row_list =[]     

    for index, rows in a.iterrows(): 
        
        my_list = [rows[i] for i in a.columns]     
        row_list.append(my_list) 
    
    target = [rows for _, rows in b.items()]
        
    return row_list, target

In [ ]:
address = "E:\\DKE related\\Studies\\Second Semester\\ATiML\\Codes\\Semester Project\\feature_vector.csv"

df = pd.read_csv(address)

# df['genre'] = df['genre'].astype('category')
# df['genre'] = df['genre'].cat.codes

# X_train,X_test,y_train,y_test = train_test_split(df.iloc[:, 1:-1].to_numpy(), df.iloc[:, -1].to_numpy(),random_state=0)

df[df['genre']=='Allegories']
df.drop([90, 93], inplace=True)

In [ ]:
X_train

In [ ]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=500).batch(50).repeat(200)

test_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_data = test_data.batch(64)

# for item in train_data:
#     print(item)

In [ ]:
from tensorflow import keras

model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(50, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))

model.fit(train_data, epochs=3)

result = model.evaluate(test_data)
print(model.metrics_names, result)

In [ ]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=X_train.shape[1:]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit(X_train, y_train, epochs=20)
# mse_test = model.evaluate(X_test, y_test)
# X_new = X_test[:3]
# y_pred = model.predict(X_new)


plt.plot(pd.DataFrame(history.history))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

y_pred

In [ ]:
df

In [ ]:
address = "E:\\DKE related\\Studies\\Second Semester\\ATiML\\Codes\\Semester Project\\feature_vector.csv"

df = pd.read_csv(address)
df['genre'] = df['genre'].astype('category')
df['genre'] = df['genre'].cat.codes

df

In [ ]:
X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

scaler = StandardScaler()
X = scaler.fit_transform(X.astype(np.float64))



In [ ]:
X_train_val, X_test, Y_train_val, Y_test = train_test_split(df.iloc[:, 1:-1], df.iloc[:, -1], test_size=0.20, random_state = 0, shuffle=True, stratify = df.iloc[:, -1])

skf = StratifiedKFold(n_splits=5, shuffle=True)

for train_index, val_index in skf.split(X_train_val, Y_train_val):
    
    X_train, X_val = X[train_index], X[val_index]
    Y_train, Y_val = Y[train_index], Y[val_index]

    print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

    nb = MultinomialNB()

    %time nb.fit(X_train, Y_train)

    y_pred_nb = nb.predict(X_val)
    acc_nb = metrics.accuracy_score(y_val, y_pred_nb)
    print(acc_nb)
    


In [ ]:
X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state = 0, shuffle=True, stratify = Y)

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


In [ ]:
nb = MultinomialNB()

%time nb.fit(X_train, Y_train)

Y_pred_nb = nb.predict(X_test)

acc_nb = metrics.accuracy_score(Y_test, Y_pred_nb)
print(acc_nb)

In [ ]:
X

In [51]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from sklearn import metrics

val_acc_svm = 0
val_acc_logistic = 0
val_acc_nb = 0
val_acc_rf = 0

X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 0, shuffle=True, stratify = Y)

skf = StratifiedKFold(n_splits=5, shuffle=True)

for train_index, val_index in skf.split(X_train_val, Y_train_val):
    
    X_train, X_val = X_train_val[train_index], X_train_val[val_index]
    Y_train, Y_val = Y_train_val[train_index], Y_train_val[val_index]
    
    X_train, Y_train = tackle_data_imbalance(X_train, Y_train) 

    val_acc_svm += perform_SVM(X_train, Y_train, X_val, Y_val)
    val_acc_logistic += perform_Logistic(X_train, Y_train, X_val, Y_val)
    val_acc_nb += perform_NB(X_train, Y_train, X_val, Y_val)
    val_acc_rf += perform_random_forrest(X_train, Y_train, X_val, Y_val)

    # print("SVM: {}; Logistic: {}; NB: {}; RB: {}".format(round(acc_svm, 4), round(acc_logistic, 4), round(acc_nb, 4), round()))

val_acc_svm = float(val_acc_svm/5)
val_acc_logistic = float(val_acc_logistic/5)
val_acc_nb = float(val_acc_nb/5) 
val_acc_rf = float(val_acc_rf/5) 

print("Average Validation Accuracy")
print("SVM: {}; Logistic: {}; NB: {}; RB: {}".format(round(val_acc_svm, 4), round(val_acc_logistic, 4), round(val_acc_nb, 4), round(val_acc_rf, 4)))

# test accuracy 


TypeError: __init__() got an unexpected keyword argument 'loss'

In [ ]:
from collections import Counter
import operator

def tackle_data_imbalance(X, Y):
    
    famous_dict = {}
    counter = Counter(Y)    
    higest_key, highest_val = max(counter.items(), key=operator.itemgetter(1))
    
    for key in counter:    
        famous_dict[key] = counter.get(key)*2
            
    over = ADASYN(n_neighbors=1, sampling_strategy=famous_dict)    
    under = RandomUnderSampler(sampling_strategy="auto")
    
    X, Y = over.fit_resample(X, Y)
    X, Y = under.fit_resample(X, Y)
    return X, Y

In [ ]:
import operator


famous_dict = {
        7: 10, 
        6: 10, 
        5: 9, 
        0: 9, 
        8: 9, 
        2: 4, 
        3: 4, 
        4: 4, 
        1: 1
    }

counter = Counter(famous_dict)

new_dict = {}
higest_key, highest_val = max(famous_dict.items(), key=operator.itemgetter(1))

new_dict = dict((key, counter.get(key)*2) for key in counter)

print(new_dict)

In [ ]:
from sklearn.svm import SVC

def perform_SVM(X_train, Y_train, X_val, Y_val):
    # SVM 
    
    svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="poly", degree=10, coef0=50, C=500))
    ])


    # ("svm_clf", SVC(kernel="rbf", gamma=0.0001, C=0.1)) -> 28 
    # ("svm_clf", SVC(kernel="poly", degree=10, coef0=50, C=500)) -> 30
    
    print("fitting data to SVM")
    # svm_clf = SVC(gamma='auto')
    %time svm_clf.fit(X_train, Y_train)
    Y_pred_svm = svm_clf.predict(X_val)
    
    return metrics.accuracy_score(Y_val, Y_pred_svm)

def perform_Logistic(X_train, Y_train, X_val, Y_val):
    
    softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
    print("fitting data to Logistic")
    %time softmax_reg.fit(X_train, Y_train)
    Y_pred_logistic = softmax_reg.predict(X_val)    
    return metrics.accuracy_score(Y_val, Y_pred_logistic)

    # Multinomial Naive Bayes

def perform_NB(X_train, Y_train, X_val, Y_val):

    nb = MultinomialNB()
    print("fitting data to Naive Bayes")
    %time nb.fit(X_train, Y_train)
    Y_pred_nb = nb.predict(X_val)
    return metrics.accuracy_score(Y_val, Y_pred_nb)

def perform_random_forrest(X_train, Y_train, X_val, Y_val):

    clf = RandomForestClassifier(max_depth=2, random_state=0)
    clf.fit(X_train, Y_train)

    Y_pred = clf.predict(X_val)
    return metrics.accuracy_score(Y_val, Y_pred)

In [ ]:
def select_k_features(k, X_train, X_test, Y_train):
    ch2 = feature_selection.SelectKBest(feature_selection.chi2, k=k)
    X_train = ch2.fit_transform(X_train, Y_train)
    X_test = ch2.transform(X_test)
    return X_train, X_test

In [ ]:
X = df.iloc[:, 1:-1].to_numpy()
Y = df.iloc[:, -1].to_numpy()

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, stratify = Y)

In [ ]:
X_train_val, X_test = select_k_features(10, X_train_val, X_test, Y_train_val)
perform_random_forrest(X_train_val, Y_train_val, X_test, Y_test)
